# MEIRL

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
area = "london"
filename = "data/" + area + "/adjacency_matrix.csv"
adjacency_matrix = pd.read_csv(filename, index_col=0)

In [3]:
expert_feature_count = adjacency_matrix.copy()
expert_feature_count["value"] = 0
expert_feature_count = expert_feature_count["value"]
expert_feature_count = expert_feature_count.to_frame()
learner_feature_count = expert_feature_count.copy()
test_feature_count = expert_feature_count.copy()

In [4]:
# cross_validation
all_data = 400
training_number = int(all_data * 0.75)
test_number = int(all_data * 0.25)

# Import expert trajectory

### prepare dataset for training

In [5]:
start_point_list = []
for i in range (1,training_number+1):
    filename = "data/" + area + "/" + str(i)
    file = open(filename, "r")
    expert_trajectory = file.read().split(', ')
    start_point_list.append(expert_trajectory[0])
    for j in expert_trajectory:
        current_count = int(expert_feature_count.loc[int(j), ["value"]].values[0])
        add_count = expert_trajectory.count(j)
        new_count = current_count + add_count 
        expert_feature_count.loc[int(j), ["value"]] = new_count
expert_feature_count = (expert_feature_count - expert_feature_count.min()) / expert_feature_count.max() - expert_feature_count.min()    

### prepare for testing

In [6]:
for i in range (training_number + 1 , all_data):
    filename = "data/" + area + "/" + str(i)
    file = open(filename, "r")
    expert_trajectory = file.read().split(', ')
    for j in expert_trajectory:
        current_count = int(test_feature_count.loc[int(j), ["value"]].values[0])
        add_count = expert_trajectory.count(j)
        new_count = current_count + add_count 
        test_feature_count.loc[int(j), ["value"]] = new_count
test_feature_count = (test_feature_count - test_feature_count.min()) / test_feature_count.max() - test_feature_count.min()    

# Let's train

In [12]:
#Initialization of reward parameter
reward_parameter = expert_feature_count.copy()
reward_parameter.columns = ["value"]
reward_parameter["value"] = np.random.random_sample(reward_parameter.shape[0])

In [13]:
def update_position(current_position, eps, reward):
    
    action_list = np.array(adjacency_matrix[adjacency_matrix[current_position] == 1].index)
    value_list = reward[reward.index.isin(action_list)]

    if np.random.random() > eps:
        
        total_reward_lists = []
        for i in range(len(action_list)):
            immediate_reward = value_list.values[i][0]
            one_step_position = value_list.index[i]
            one_step_action_list = np.array(adjacency_matrix[adjacency_matrix[str(one_step_position)] == 1].index)
            one_step_value_list = reward[reward.index.isin(one_step_action_list)]
            for j in range(len(one_step_value_list)):
                total_reward_list = []
                one_step_reward = one_step_value_list.values[j][0]
                total_reward = immediate_reward + one_step_reward
                total_reward_list = [total_reward, i, j]
                total_reward_lists.append(total_reward_list)
        total_reward_lists = np.array(total_reward_lists)
        highest_reward = np.argmax(total_reward_lists[:,0])
        updated_position = action_list[int(total_reward_lists[highest_reward][1])]

    else:
        updated_position = random.choice(value_list["value"].index.tolist())

    return updated_position

In [14]:
training_number[]

SyntaxError: invalid syntax (<ipython-input-14-020b1c6b3062>, line 1)

In [15]:
# how many iterations?
Y = []
iteration = 300
for i in range (iteration):
    epsilon = 1 / (i+1)  
    param = 1 / (i+1)
    learner_feature_count["value"] = 0
    for j in range (1,training_number):
        current_position = start_point_list[j]
        # agent will walk for 50 steps
        for k in range(50):
            current_position = str(update_position(current_position, epsilon, reward_parameter))
            learner_feature_count.loc[int(current_position), ["value"]] += 1
    
    learner_feature_count = (learner_feature_count - learner_feature_count.min()) / (learner_feature_count.max() - learner_feature_count.min() )   
    gradient = (expert_feature_count - learner_feature_count) 
    reward_parameter = (gradient*param) + reward_parameter
    reward_parameter = (reward_parameter - reward_parameter.min()) / (reward_parameter.max() - reward_parameter.min() )   
    
    difference = gradient.abs().sum().values[0]
    Y.append(difference)
    print(i, difference)
    
    correlation_dataframe = pd.concat([learner_feature_count, test_feature_count], axis=1)
    correlation = correlation_dataframe.corr().values[0][1]
    print(correlation)

0 193.895491803
0.0573695799713
1 132.028010033
0.137993795437
2 104.567307692
0.213638260861
3 107.011571056
0.304905448804
4 106.636248186
0.320510259386
5 105.240384615
0.31784573297
6 101.828807007
0.382361739392
7 106.850243148
0.325167074873
8 101.998160233
0.378579467725
9 103.208435083
0.422324121196
10 102.507695798
0.370077260497
11 103.495020045
0.321244257686
12 103.511379675
0.403513575693
13 103.78332259
0.400262475357
14 101.064736734
0.442142438886
15 101.833624709
0.422916321387
16 102.475921992
0.411324992842
17 104.814150655
0.39651893855
18 106.716460623
0.35908285634
19 105.276599503
0.438528555154
20 105.014741722
0.46401519622
21 105.051341936
0.405565229363
22 102.05928391
0.40083230037
23 103.949498567
0.394118105554
24 103.223091667
0.456451359731
25 102.002076923
0.465522655376
26 108.095489188
0.363397222572
27 103.461982968
0.403058759223
28 103.330570292
0.457910410576
29 105.969556037
0.383302030618
30 104.391678012
0.402890602975
31 102.154047619
0.47854

254 105.249496816
0.39166379131
255 105.314920125
0.413365163166
256 106.006108597
0.387587904209
257 103.267820513
0.44014013409
258 104.209583748
0.404152396436
259 107.994309262
0.313298687375
260 106.209279476
0.336347244618
261 104.346789031
0.378353763857
262 105.993211343
0.354146485408
263 104.524646331
0.418349730965
264 105.907908437
0.334623711822
265 106.108184965
0.349230196711
266 104.937567714
0.346433112819
267 105.728213622
0.403466622036
268 106.121245927
0.393054137599
269 107.301841135
0.343778564304
270 105.019016593
0.372845861889
271 105.967123077
0.357425374987
272 106.917633072
0.368029022237
273 104.169311057
0.409663439001
274 105.697739137
0.395463425074
275 104.162300888
0.351587317912
276 106.237395944
0.378800831899
277 107.543359007
0.348203576943
278 105.391619183
0.40717977098
279 107.112161506
0.360718564961
280 105.530561232
0.400721452673
281 105.244122257
0.42915989907
282 106.881567242
0.354436451701
283 106.355884385
0.381575686509
284 104.896995

# Local Depth

In [7]:
#local depth
filename = "data/" + area + "/" + "closeness_centrality.csv"

closeness_centrality = pd.read_csv(filename, index_col=0)
adjacent_node = adjacency_matrix.sum().to_frame()
adjacent_node.columns = ["closeness_centrality"]
adjacent_node.index = adjacent_node.index.map(int)
mean_depth = (closeness_centrality - adjacent_node) / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   

In [8]:
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_local_depth = - correlation_dataframe.corr().values[0][1]

In [9]:
correlation_local_depth

0.45406804231592879

# Connectivity 

In [10]:
mean_depth = adjacent_node / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_connectivity = correlation_dataframe.corr().values[0][1]
correlation_connectivity

0.32460389310488996

# Global Depth

In [11]:
mean_depth = closeness_centrality / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_global_depth = - correlation_dataframe.corr().values[0][1]
correlation_global_depth

0.45406704994670616